In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.parallel
import pandas as pd
import tensorflow as tf
#from ray import tune
#from ray.tune import grid_search, uniform
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import argparse
import scipy.io
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
mat_data = scipy.io.loadmat('drive/MyDrive/Porsche/7000_scaled_2LVL_properties_without_Udc_nocycle_data.mat')
#mat_data2 = scipy.io.loadmat('drive/MyDrive/Porsche/7000_data_entries_per_design.mat')

#Get scaled input and scaled output data
#Min max Scaling
input = mat_data.get('input')
output = mat_data.get('output')
input_scaled = mat_data.get('input_scaled')
output_scaled = mat_data.get('output_scaled')
#data_entries = mat_data2.get('data_entries_per_design_7000_input')

# Convert the input data to a NumPy array
input_data = np.array(input)
output_data = np.array(output)
#data_entries = np.array(data_entries)

'''
num_data_per_design = 1502
num_designs = data_entries.shape[0]

# Create an array to store the reduced input and output data
reduced_input_data = np.empty((num_designs * num_data_per_design, input_data.shape[1]))
reduced_output_data = np.empty((num_designs * num_data_per_design, output_data.shape[1]))

# Generate reduced data by randomly sampling from each design
start_index = 0
for i in range(num_designs):
    num_rows = data_entries[i][0]  # Access the value from data_entries array
    design_data = input_data[start_index : start_index + num_rows]
    design_output = output_data[start_index : start_index + num_rows]
    if num_rows > num_data_per_design:
        indices = np.random.choice(num_rows, num_data_per_design, replace=False)
        design_data = design_data[indices]
        design_output = design_output[indices]
    reduced_input_data[start_index : start_index + design_data.shape[0], :] = design_data
    reduced_output_data[start_index : start_index + design_output.shape[0], :] = design_output
    start_index += design_data.shape[0]

# Save the reduced input data as a MATLAB .mat file
scipy.io.savemat('reduced_input_data.mat', {'reduced_input_data': reduced_input_data})
'''

"\nnum_data_per_design = 1502\nnum_designs = data_entries.shape[0]\n\n# Create an array to store the reduced input and output data\nreduced_input_data = np.empty((num_designs * num_data_per_design, input_data.shape[1]))\nreduced_output_data = np.empty((num_designs * num_data_per_design, output_data.shape[1]))\n\n# Generate reduced data by randomly sampling from each design\nstart_index = 0\nfor i in range(num_designs):\n    num_rows = data_entries[i][0]  # Access the value from data_entries array\n    design_data = input_data[start_index : start_index + num_rows]\n    design_output = output_data[start_index : start_index + num_rows]\n    if num_rows > num_data_per_design:\n        indices = np.random.choice(num_rows, num_data_per_design, replace=False)\n        design_data = design_data[indices]\n        design_output = design_output[indices]\n    reduced_input_data[start_index : start_index + design_data.shape[0], :] = design_data\n    reduced_output_data[start_index : start_index + d

In [3]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 2048) #64
        self.bn1 = nn.BatchNorm1d(2048)  # Batch normalization layer
        self.fc2 = nn.Linear(2048, 1024)
        self.bn2 = nn.BatchNorm1d(1024)  # Batch normalization layer
        self.fc3 = nn.Linear(1024, 512)
        self.bn3 = nn.BatchNorm1d(512)
        self.fc4 = nn.Linear(512, 256)
        self.bn4 = nn.BatchNorm1d(256)
        #self.fc5 = nn.Linear(256, 128)
        #self.bn5 = nn.BatchNorm1d(128)
        self.fc6 = nn.Linear(256, output_size)
        self.reakylu = nn.LeakyReLU()
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.elu = nn.ELU()
        self.tanh = nn.Tanh()

        #Dropout some neurons for overfiitng
        self.dropout = nn.Dropout(0.2)



    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.dropout(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.dropout(x)
        x = self.relu(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = self.dropout(x)
        x = self.relu(x)

        x = self.fc4(x)
        x = self.bn4(x)
        x = self.dropout(x)
        x = self.relu(x)

       # x = self.fc5(x)
       # x = self.bn5(x)
       # x = self.dropout(x)
       # x = self.relu(x)

        x = self.fc6(x)
        x = self.sigmoid(x)
        return x

In [4]:
# Set device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
input_data = torch.Tensor(input_scaled)
output_data = torch.Tensor(output_scaled)
#print(input_data)


# Split the data into training, validation, and testing sets
#test_size = 0.2 ist 80% Training set
train_data, test_data, train_target, test_target = train_test_split(
    input_data, output_data, test_size=0.3, random_state=42
)

#test_size hier auf 0.5 heißt die übrigen 20% nochmal in 10% für val und 10% für test
val_data, test_data, val_target, test_target = train_test_split(
    test_data, test_target, test_size=0.5, random_state=42
)
#data = pd.DataFrame(train_data.numpy())
#print(len(data.iloc[:,7]))
#unique_values = data.iloc[:,7].unique()
#print(len(unique_values))
# Create data loaders
train_dataset = TensorDataset(train_data, train_target)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(val_data, val_target)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_dataset = TensorDataset(test_data, test_target)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


cuda


In [5]:
# Define the dimensions of your input, hidden, and output layers
input_size = input_data.shape[1]
output_size = output_data.shape[1]
print(input_data.shape[0])
# Create an instance of the MLP model
model = MLP(input_size, output_size).to(device)
#model = MLP1(input_size, output_size).to(device)

#Define inital data
test_accuracies = []
best_test_accuracy = 0.0
best_val_loss = float('inf')

#Get dropout_value
num = ""
#get_dropout = MLP(input_size, output_size).dropout
get_dropout = MLP(input_size, output_size).dropout
dropout_number = str(get_dropout)
for c in dropout_number:
    if c.isdigit():
        num = num + c
print('Dropout value: ' + num)

# Define the loss function and optimizer
str_MLP = 0
lr = 0.001
string_loss = "MSE"
string_optimizer = "Adam"
num_epochs = 8
train_losses = []
val_losses = []

2033336
Dropout value: 02


In [ ]:
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from torch.utils.data import Subset

# MSE Loss function
criterion = nn.MSELoss()
# MAE Loss function
# criterion = nn.L1Loss()
# criterion = nn.SmoothL1Loss()
# optimizer = optim.RMSprop(model.parameters(), lr)
optimizer = optim.Adam(model.parameters(), lr)
# optimizer = optim.Adamax(model.parameters(), lr)
# optimizer = optim.SGD(model.parameters(), lr)

num_folds = 10
r2_scores = []
test_losses = []

kf = KFold(n_splits=num_folds, shuffle=True)

for train_index, test_index in kf.split(train_dataset):
    train_loader = DataLoader(
        Subset(train_dataset, train_index),
        batch_size=64,
        shuffle=True
    )
    test_loader = DataLoader(
        Subset(train_dataset, test_index),
        batch_size=64,
        shuffle=False
    )

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        train_loss = 0.0
        test_loss = 0.0

        # Training
        model.train()
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        # Testing
        model.eval()
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

        test_loss /= len(test_loader)
        test_losses.append(test_loss)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.6f}")

    # Calculate R2 score for test set
    test_predictions = []
    test_labels = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            test_predictions.append(outputs.cpu().numpy())
            test_labels.append(labels.cpu().numpy())

    test_predictions = np.concatenate(test_predictions, axis=0)
    test_labels = np.concatenate(test_labels, axis=0)
    test_r2 = r2_score(test_labels, test_predictions)
    r2_scores.append(test_r2)

average_r2 = np.mean(r2_scores)
print(f"Average R^2 Score: {average_r2:.4f}")
for i in range(len(r2_scores)):
  print(f"This were all R^2 Scores: {r2_scores[i]:.4f}")

Epoch [1/8], Train Loss: 0.000343, Test Loss: 0.000039
